In [1]:
from transformers import PreTrainedTokenizerFast, GenerationConfig , PhiForCausalLM, pipeline
import torch

# 模型保存路径
这里加载sft后的模型和dpo后的模型，看看两者输出的区别

In [2]:
tokenizer_dir = './model_save/tokenizer/'
dpo_model_save_dir = './model_save/dpo/'
sft_model_save_dir = './model_save/sft/'
max_seq_len = 320

## 加载模型

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [4]:
tokenizer = PreTrainedTokenizerFast.from_pretrained(tokenizer_dir)
dpo_model = PhiForCausalLM.from_pretrained(dpo_model_save_dir).to(device)

sft_mdodel = PhiForCausalLM.from_pretrained(sft_model_save_dir).to(device)

model_size = sum(t.numel() for t in dpo_model.parameters())
print(f"Phi-2 size: {model_size / 1000**2:.1f}M parameters")

Phi-2 size: 193.7M parameters


## 定义两个text-generation的pipeline

In [5]:
dpo_pipe = pipeline(
    "text-generation", 
    model=dpo_model, 
    device=device,
    tokenizer=tokenizer,
)

sft_pipe = pipeline(
    "text-generation", 
    model=sft_mdodel, 
    device=device,
    tokenizer=tokenizer,
)

## 模型输出

In [6]:
txt = '感冒了要怎么办？'
prompt = f"##提问:\n{txt}\n##回答:\n"
sft_outputs = sft_pipe(prompt, num_return_sequences=1, max_new_tokens=256, pad_token_id=tokenizer.eos_token_id)
dpo_outputs = dpo_pipe(prompt, num_return_sequences=1, max_new_tokens=256, pad_token_id=tokenizer.eos_token_id)


print(f"model after sft output:\n\n{sft_outputs[0]['generated_text']}")
print('\n==================\n')
print(f"model after dpo output:\n\n{dpo_outputs[0]['generated_text']}")

model after sft output:

##提问:
感冒了要怎么办？
##回答:
感冒时最好休息，多喝水，避免吃辛辣刺激的食物，如辛辣食物，以及保持室内空气流通。


model after dpo output:

##提问:
感冒了要怎么办？
##回答:
感冒是由病毒引起的，感冒一般由病毒引起，以下是一些常见感冒的方法：
- 洗手，特别是在接触其他人或物品后。
- 咳嗽或打喷嚏时用纸巾或手肘遮住口鼻。
- 触摸面部、鼻子和眼睛。
- 喝充足的水，特别是在感冒季节。
- 喝一些温水或鸡汤，以帮助身体降温。
- 如果感冒症状严重，如发热或咳嗽，可能需要就医。


## 不用pipeline组件，使用greedy search方法手动生成

In [7]:
# greedy search
gen_conf = GenerationConfig(
    num_beams=1,
    do_sample=False,
    max_length=320,
    max_new_tokens=256,
    no_repeat_ngram_size=4,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
)

In [8]:
tokend = tokenizer.encode_plus(text=prompt)
input_ids, attention_mask = torch.LongTensor([tokend.input_ids]).to(device), torch.LongTensor([tokend.attention_mask]).to(device)

outputs = dpo_model.generate(
    inputs=input_ids,
    attention_mask=attention_mask,
    generation_config=gen_conf,
)

outs = tokenizer.decode(outputs[0].cpu().numpy(), clean_up_tokenization_spaces=True, skip_special_tokens=True,)
print(outs)

##提问:
感冒了要怎么办？
##回答:
感冒是由病毒引起的，感冒一般由病毒引起，以下是一些常见感冒的方法：
- 洗手，特别是在接触其他人或物品后。
- 咳嗽或打喷嚏时用纸巾或手肘遮住口鼻。
- 用手触摸口鼻，特别是喉咙和鼻子。
- 如果咳嗽或打喷嚏，可以用纸巾或手绢来遮住口鼻，但要远离其他人。
- 如果你感冒了，最好不要触摸自己的眼睛、鼻子和嘴巴。
- 在感冒期间，最好保持充足的水分和休息，以缓解身体的疲劳。
- 如果您已经感冒了，可以喝一些温水或盐水来补充体液。
- 另外，如果感冒了，建议及时就医。
